[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/lisphilar/covid19-sir/HEAD?labpath=example%2F02_data_engineering.ipynb)

# Data engineering
With [Data preparation tutorial](https://github.com/lisphilar/covid19-sir/blob/master/example/01_data_preparation.ipynb), we prepared datasets (geospatial time-series data) to analyze.  As the next step of data engineering, we will perform the followings here.

1. Data cleaning
2. Data transformation
3. Arithmetic operations
4. EDA at a geospatial layer
5. Data subsetting for a location and data complement
6. EDA of subset

Note that EDA = explanatory data analysis

In [25]:
from pprint import pprint
import covsirphy as cs
import numpy as np
cs.__version__

'3.0.0'

We will use the recommended datasets at country-level data as an example.

In [26]:
eng = cs.DataEngineer()
eng.download(databases=["japan", "covid19dh", "owid"])
eng.all().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276931 entries, 0 to 276930
Data columns (total 27 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   ISO3                                 276931 non-null  category      
 1   Province                             276931 non-null  category      
 2   City                                 276931 non-null  category      
 3   Date                                 276931 non-null  datetime64[ns]
 4   Cancel_events                        197480 non-null  Float64       
 5   Confirmed                            237857 non-null  Float64       
 6   Contact_tracing                      197507 non-null  Float64       
 7   Country                              270981 non-null  string        
 8   Fatal                                220660 non-null  Float64       
 9   Gatherings_restrictions              197480 non-null  Float64       
 

## 1. Data cleaning
`DataEngineer().clean()` performs the following data cleaning functionalities. By applying a list of strings to `kinds` argument (eg. `kinds=["resample"]`), we can specify the cleaning method(s).


* "convert_date": Convert dtype of date column to pandas.Timestamp.
* "resample": Resample records with dates.
* "fillna": Fill NA values with '-' (layers) and the previous values and 0.

For "convert_date", keyword arguments of pandas.to_datetime() including "dayfirst (bool): whether date format is DD/MM or not" can be used.

For "resample", `date_range=<tuple of (str or None, str or None) or None>)` can be applied as keyword arguments to set the range.

In [27]:
eng.clean()
eng.all().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277031 entries, 0 to 277030
Data columns (total 27 columns):
 #   Column                               Non-Null Count   Dtype         
---  ------                               --------------   -----         
 0   ISO3                                 277031 non-null  category      
 1   Province                             277031 non-null  category      
 2   City                                 277031 non-null  category      
 3   Date                                 277031 non-null  datetime64[ns]
 4   Cancel_events                        277031 non-null  Float64       
 5   Confirmed                            277031 non-null  Float64       
 6   Contact_tracing                      277031 non-null  Float64       
 7   Country                              277031 non-null  object        
 8   Fatal                                277031 non-null  Float64       
 9   Gatherings_restrictions              277031 non-null  Float64       
 

## 2. Data transformation
Transform all registered data, calculating the number of susceptible and infected cases. This is required to analyze real data with SIR-derived models.

- Susceptible = Population - Confirmed
- Infected = Confirmed - Fatal - Recovered

In [28]:
main_variables = ["Population", "Susceptible", "Confirmed", "Infected", "Fatal", "Recovered"]

In [29]:
eng.transform()
eng.all(variables=main_variables).tail()

,ISO3,Province,City,Date,Population,Susceptible,Confirmed,Infected,Fatal,Recovered
277026,ZWE,-,-,2023-03-05,14439018.0,14174891.0,264127.0,175465.0,5668.0,82994.0
277027,ZWE,-,-,2023-03-06,14439018.0,14174891.0,264127.0,175465.0,5668.0,82994.0
277028,ZWE,-,-,2023-03-07,14439018.0,14174891.0,264127.0,175465.0,5668.0,82994.0
277029,ZWE,-,-,2023-03-08,14439018.0,14174742.0,264276.0,175611.0,5671.0,82994.0
277030,ZWE,-,-,2023-03-09,14439018.0,14174742.0,264276.0,175611.0,5671.0,82994.0


Recalculation of "Population" and "Confirmed" can be performed with `DataEngineer().inverse_transform()`, if necessary. (No impact with this example data.)

In [30]:
eng.inverse_transform()
eng.all(variables=main_variables).tail()

,ISO3,Province,City,Date,Population,Susceptible,Confirmed,Infected,Fatal,Recovered
277026,ZWE,-,-,2023-03-05,14439018,14174891.0,264127,175465.0,5668.0,82994.0
277027,ZWE,-,-,2023-03-06,14439018,14174891.0,264127,175465.0,5668.0,82994.0
277028,ZWE,-,-,2023-03-07,14439018,14174891.0,264127,175465.0,5668.0,82994.0
277029,ZWE,-,-,2023-03-08,14439018,14174742.0,264276,175611.0,5671.0,82994.0
277030,ZWE,-,-,2023-03-09,14439018,14174742.0,264276,175611.0,5671.0,82994.0


## 3. Arithmetic operations
We can perform arithmetic operations to add new columns.

* `.diff(column, suffix="_diff", freq="D")`: Calculate daily new cases with "f(x>0) = F(x) - F(x-1), x(0) = 0 when F is cumulative numbers".
* `.add(columns, new=None, fill_value=0)`: Calculate element-wise addition with pandas.DataFrame.sum(axis=1), X1 + X2 + X3 +...
* `.mul(columns, new=None, fill_value=0)`: Calculate element-wise multiplication with pandas.DataFrame.product(axis=1), X1 * X2 * X3 *...
* `.sub(minuend, subtrahend, new=None, fill_value=0)`: Calculate element-wise subtraction with pandas.Series.sub(), minuend - subtrahend.
* `.div(columns, new=None, fill_value=0)`: Calculate element-wise floating division with pandas.Series.div(), numerator / denominator.
* `.assign(**kwargs))`: Assign a new column with pandas.DataFrame.assign().


In [31]:
# Diff
eng.diff(column="Confirmed", suffix="_diff", freq="D")
eng.all(variables=["Confirmed", "Confirmed_diff"]).tail()

,ISO3,Province,City,Date,Confirmed,Confirmed_diff
277026,ZWE,-,-,2023-03-05,264127,0
277027,ZWE,-,-,2023-03-06,264127,0
277028,ZWE,-,-,2023-03-07,264127,0
277029,ZWE,-,-,2023-03-08,264276,149
277030,ZWE,-,-,2023-03-09,264276,0


In [32]:
# Addition
eng.add(columns=["Fatal", "Recovered"])
eng.all(variables=["Fatal", "Recovered", "Fatal+Recovered"]).tail()

,ISO3,Province,City,Date,Fatal,Recovered,Fatal+Recovered
277026,ZWE,-,-,2023-03-05,5668.0,82994.0,88662
277027,ZWE,-,-,2023-03-06,5668.0,82994.0,88662
277028,ZWE,-,-,2023-03-07,5668.0,82994.0,88662
277029,ZWE,-,-,2023-03-08,5671.0,82994.0,88665
277030,ZWE,-,-,2023-03-09,5671.0,82994.0,88665


In [33]:
# Multiplication
eng.mul(columns=["Confirmed", "Recovered"])
eng.all(variables=["Confirmed", "Recovered", "Confirmed*Recovered"]).tail()

,ISO3,Province,City,Date,Confirmed,Recovered,Confirmed*Recovered
277026,ZWE,-,-,2023-03-05,264127,82994.0,21920956238.0
277027,ZWE,-,-,2023-03-06,264127,82994.0,21920956238.0
277028,ZWE,-,-,2023-03-07,264127,82994.0,21920956238.0
277029,ZWE,-,-,2023-03-08,264276,82994.0,21933322344.0
277030,ZWE,-,-,2023-03-09,264276,82994.0,21933322344.0


In [34]:
# Division
eng.div(numerator="Confirmed", denominator="Tests", new="Positive_rate")
# Assignment of new a new column
eng.assign(**{"Positive_rate_%": lambda x: x["Positive_rate"] * 100})
eng.all(variables=["Tests", "Confirmed", "Positive_rate_%"]).tail()

,ISO3,Province,City,Date,Tests,Confirmed,Positive_rate_%
277026,ZWE,-,-,2023-03-05,2379907.0,264127,11.098207
277027,ZWE,-,-,2023-03-06,2379907.0,264127,11.098207
277028,ZWE,-,-,2023-03-07,2379907.0,264127,11.098207
277029,ZWE,-,-,2023-03-08,2379907.0,264276,11.104468
277030,ZWE,-,-,2023-03-09,2379907.0,264276,11.104468


## 4. EDA at a geospatial layer
`DataEngineer().layer()` returns the data at the selected layer in the date range.

Arguments:

* geo (tuple(list[str] or tuple(str) or str) or str or None): location names to specify the layer or None (the top level)
* start_date (str or None): start date, like 22Jan2020
* end_date (str or None): end date, like 01Feb2020
* variables (list[str] or None): list of variables to add or None (all available columns)

In [35]:
eng.layer().query("ISO3 == 'USA'").tail()

,ISO3,Province,City,Date,Cancel_events,Confirmed,Confirmed*Recovered,Confirmed_diff,Contact_tracing,Country,...,Stringency_index,Susceptible,Testing_policy,Tests,Transport_closing,Vaccinated_full,Vaccinated_once,Vaccinations,Vaccinations_boosters,Workplace_closing
260253,USA,-,-,2023-06-17,0.0,103910034,0.0,0,1.0,United States,...,37.04,222777467.0,3.0,912769124.0,0.0,230637348.0,270227181.0,676728782.0,133062763.0,-1.0
260254,USA,-,-,2023-06-18,0.0,103910034,0.0,0,1.0,United States,...,37.04,222777467.0,3.0,912769124.0,0.0,230637348.0,270227181.0,676728782.0,133062763.0,-1.0
260255,USA,-,-,2023-06-19,0.0,103910034,0.0,0,1.0,United States,...,37.04,222777467.0,3.0,912769124.0,0.0,230637348.0,270227181.0,676728782.0,133062763.0,-1.0
260256,USA,-,-,2023-06-20,0.0,103910034,0.0,0,1.0,United States,...,37.04,222777467.0,3.0,912769124.0,0.0,230637348.0,270227181.0,676728782.0,133062763.0,-1.0
260257,USA,-,-,2023-06-21,0.0,103910034,0.0,0,1.0,United States,...,37.04,222777467.0,3.0,912769124.0,0.0,230637348.0,270227181.0,676728782.0,133062763.0,-1.0


This dataset has only country-level data and `geo` should be country name. We can select the followings as `geo` argument for EDA at a geospatial layer when we have adequate data.

* When `geo=None` or `geo=(None,)`, returns country-level data, assuming we have country/province/city as layers here.
* When `geo=("Japan",)` or `geo="Japan"`, returns province-level data in Japan.
* When `geo=(["Japan", "UK"],)`, returns province-level data in Japan and UK.
* When `geo=("Japan", "Kanagawa")`, returns city-level data in Kanagawa/Japan.
* When `geo=("Japan", ["Tokyo", "Kanagawa"])`, returns city-level data in Tokyo/Japan and Kanagawa/Japan.

Additionally, we can create a choropleth map with a data at a geospatial layer on a date.

arguments of `DataEngineer().choropleth()`:

* geo (tuple(list[str] or tuple(str) or str) or str or None): location names to specify the layer or None (the top level)
* variable (str): variable name to show
* on (str or None): the date, like 22Jan2020, or None (the last date of each location)
* title (str): title of the map
* filename (str or None): filename to save the figure or None (display)
* logscale (bool): whether convert the value to log10 scale values or not
* directory (str): directory to save GeoJSON file of "Natural Earth" GitHub repository
* natural_earth (str or None): title of GeoJSON file (without extension) of "Natural Earth" GitHub repository or None (automatically determined)
* **kwargs: keyword arguments of the following classes and methods.
    * matplotlib.pyplot.savefig(), matplotlib.pyplot.legend(), and
    * pandas.DataFrame.plot()

In [36]:
from pathlib import Path
import warnings
import geopandas as gpd
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
from covsirphy.util.validator import Validator
from covsirphy.visualization.vbase import VisualizeBase

class _ChoroplethMap(VisualizeBase):
    """Create choropleth map.

    Args:
        filename (str or None): filename to save the figure or None (display)
        kwargs: the other arguments of matplotlib.pyplot.savefig()
    """

    def __init__(self, filename=None, **kwargs):
        super().__init__(filename=filename, **kwargs)
        self._geo_dirpath = Path("input")

    def __enter__(self):
        return super().__enter__()

    def __exit__(self, *exc_info):
        return super().__exit__(*exc_info)

    def plot(self, data, logscale, **kwargs):
        """Set geopandas.GeoDataFrame to create a choropleth map.

        Args:
            data (geopandas.GeoDataFrame):
                Index:
                    - reset index
                Columns:
                    - Location (str): location names
                    - Variable (str): variable to show
                    - geometry: geometric information
            logscale (bool): whether convert the value to log10 scale values or not
            kwargs: arguments of geopandas.GeoDataFrame.plot() except for 'column'
        """
        warnings.filterwarnings("ignore", category=UserWarning)
        df = Validator(data, "data").dataframe(columns=["Location", "Variable", "geometry"])
        df["Variable"] = df["Variable"].astype("float64")
        gdf = gpd.GeoDataFrame(df)
        # Color bar
        divider = make_axes_locatable(self._ax)
        cax = divider.append_axes("bottom", size="5%", pad=0.1)
        # Arguments of plotting with GeoPandas
        plot_kwargs = {
            "legend": True,
            "cmap": "coolwarm",
            "ax": self._ax,
            "cax": cax,
            "missing_kwds": {
                "color": "lightgrey",
                "edgecolor": "white",
                "hatch": "///",
            },
            "legend_kwds": {"orientation": "horizontal"},
            **kwargs
        }
        # Convert to log10 scale
        if logscale:
            gdf["Variable"] = np.log10(gdf["Variable"] + 1)
            plot_kwargs["legend_kwds"]["label"] = "in log10 scale"
        # Plotting
        if not gdf["Variable"].isna().sum():
            # Missing values are not included
            plot_kwargs.pop("missing_kwds")
        gdf.plot(column="Variable", **plot_kwargs)
        # Remove all ticks
        self._ax.tick_params(labelbottom=False, labelleft=False, left=False, bottom=False)


In [37]:
eng.choropleth(geo=None, variable="Confirmed", title="Choropleth map: Confirmed on the last date of records", filename=None)

PermissionError: [WinError 5] Access is denied: 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\covsirphy\\gis\\Natural_Earth'

## 5. Data subsetting for a location and data complement
The dataset is a geospatial time-series data. By selecting a location, the dataset will be converted to a time-series data, which is easier to analyze.

### 5.1 Subsetting
We will create a subset for selected location (eg. country, province/prefecture/state, city). Because the loaded dataset has country-level data, total values in United Kingdom (UK) on dates will be created here as an example.

Arguments of `DataEngineer().subset()`:

* geo (tuple(list[str] or tuple(str) or str) or str or None): location names to filter or None (total at the top level)
* start_date (str or None): start date, like 22Jan2020
* end_date (str or None): end date, like 01Feb2020
* variables (list[str] or None): list of variables to add or None (all available columns)
* complement (bool): whether perform data complement or not, True as default
* get_dummies (bool): whether convert categorical variable into dummy variables or not, True as default
* **Kwargs: keyword arguments for complement and default values
    * recovery_period (int): expected value of recovery period [days], 17
    * interval (int): expected update interval of the number of recovered cases [days], 2
    * max_ignored (int): Max number of recovered cases to be ignored [cases], 100
    * max_ending_unupdated (int): Max number of days to apply full complement, where max recovered cases are not updated [days], 14
    * upper_limit_days (int): maximum number of valid partial recovery periods [days], 90
    * lower_limit_days (int): minimum number of valid partial recovery periods [days], 7
    * upper_percentage (float): fraction of partial recovery periods with value greater than upper_limit_days, 0.5
    * lower_percentage (float): fraction of partial recovery periods with value less than lower_limit_days, 0.5

NOTE:  
We can select the followings as `geo` argument for subsetting when we have adequate data.

* When `geo=None` or `geo=(None,)`, returns global scale records (total values of all country-level data), assuming we have country/province/city as layers here.
* When `geo=("Japan",)` or `geo="Japan"`, returns country-level data in Japan.
* When `geo=(["Japan", "UK"],)`, returns country-level data of Japan and UK.
* When `geo=("Japan", "Tokyo")`, returns province-level data of Tokyo/Japan.
* When `geo=("Japan", ["Tokyo", "Kanagawa"])`, returns total values of province-level data of Tokyo/Japan and Kanagawa/Japan.
* When `geo=("Japan", "Kanagawa", "Yokohama")`, returns city-level data of Yokohama/Kanagawa/Japan.
* When `geo=(("Japan", "Kanagawa", ["Yokohama", "Kawasaki"])`, returns total values of city-level data of Yokohama/Kanagawa/Japan and Kawasaki/Kanagawa/Japan.

In [ ]:
# Without complement
without_df, status, status_dict = eng.subset(geo="UK", complement=False)
print(f"{status}\n")
pprint(status_dict)
cs.line_plot(without_df[["Confirmed", "Fatal", "Recovered"]], title="UK: records WITHOUT complement")

In [ ]:
# With complement (default)
with_df, status, status_dict = eng.subset(geo="Japan", complement=True)
print(f"{status}\n")
pprint(status_dict)
cs.line_plot(with_df[["Confirmed", "Fatal", "Recovered"]], title="UK: records WITH complement")

In [ ]:
with_df.info()

### 5.2 Details of data complement
When `complement=True` (default), data complement will be performed for confirmed/fatal/recovered data. This may be required for analysis because reported cumulative values sometimes show decreasing by changing accidentally for an example. Additionally, some countries, including UK, do not report the number of recovered cases.

The possible kinds of complement for each country are the following:

* "Monotonic_confirmed/fatal/recovered" (monotonic increasing complement) Force the variable show monotonic increasing.
* "Full_recovered" (full complement of recovered data) Estimate the number of recovered cases using the value of estimated average recovery period.
* "Partial_recovered" (partial complement of recovered data) When recovered values are not updated for some days, extrapolate the values.

### 5.3 Recovery period
What is "recovery period"?  
We defined "recovery period" as the time period between case confirmation and recovery (as it is subjectively defined per country). We can estimate the mode value of recovery period with class method `DataEngineer.recovery_period(data)`.

In [ ]:
jpn_df, *_ = eng.subset(geo="Japan", variables=["Confirmed", "Fatal", "Recovered"], complement=False)
recovery_period = cs.DataEngineer.recovery_period(data=jpn_df)
print(f"Mode value of recovery period in Japan: {recovery_period} [days]")

Details of recovery period calculation:

In [ ]:
df = jpn_df.resample("D").sum()
df["diff"] = df["Confirmed"] - df["Fatal"]
df = df.loc[:, ["diff", "Recovered"]].unstack().reset_index()
df.columns = ["Variable", "Date", "Number"]
df["Days"] = (df["Date"] - df["Date"].min()).dt.days
df = df.pivot_table(values="Days", index="Number", columns="Variable")
df = df.interpolate(limit_area="inside").dropna().astype(np.int64)
df["Elapsed"] = df["Recovered"] - df["diff"]
df = df.loc[df["Elapsed"] > 0]
# Calculate mode value
mode_value = round(df["Elapsed"].mode().mean())
df["Elapsed"].plot.hist(title=f"Histogram of elapsed days of recovery, mode value: {mode_value} days");

### 5.4 Alias of subsets
We can register alias names of subsets with `DataEngineer().subset_alias()`.

Arguments:  
* alias (str or None): alias name or None (list-up alias names)
* update (bool): force updating the alias when @alias is not None
* **kwargs: keyword arguments of covsirphy.DataEngineer().subset()

In [ ]:
# Register
sub1, *_ = eng.subset_alias(alias="UK_with", geo="UK", complement=True)
# Retrieve with alias
sub2, *_ = eng.subset_alias(alias="UK_with")
# Comparison
sub1.equals(sub2)

## 7. EDA of subset
With explanatory data analysis, we will get the figure of datasets.

### 7.1 Alias of variables
We can specify variables with alias. For example, "CIFR" is equivalent to list `['Confirmed', 'Infected', 'Recovered', 'Fatal']`.

In [ ]:
eng.subset(geo="Japan", variables="CIRF")[0].tail()

All aliases can be checked with `DataEngineer().variables_alias()`.

In [ ]:
eng.variables_alias()

We can register new alias "p" with `["Tests", "Confirmed", "Positive_rate_%"]` as an example.

In [ ]:
# Register new alias
eng.variables_alias(alias="p", variables=["Tests", "Confirmed", "Positive_rate_%"])
# Check the contents of an alias
eng.variables_alias(alias="p")
# Subsetting with the variable alias
eng.subset_alias(alias="jp", geo="Japan", variables="p")[0].tail()

### 7.2 Line plot
Show data with line plot. We can select function `line_plot` or class `LinePlot`.

In [ ]:
line_df, *_ = eng.subset_alias(alias="jp")

With function:

In [ ]:
cs.line_plot(
    line_df["Positive_rate_%"],
    title="Positive rate % in Japan",
    ylabel="Positive rate %",
    math_scale=False,
    show_legend=False,
    filename=None,
)

With class:

In [ ]:
with cs.LinePlot(filename=None) as lp:
    lp.plot(line_df["Positive_rate_%"])
    lp.title = "Positive rate % in Japan"
    lp.x_axis(xlabel=None)
    lp.y_axis(ylabel="Positive rate %", math_scale=False)
    lp.legend_hide()

### 7.3 Scatter plot
Show data with scatter plot. We can select function `scatter_plot` or class `ScatterPlot`.

In [ ]:
sc_df, *_ = eng.subset_alias(alias="jp")
sc_df.rename(columns={"Tests": "x", "Confirmed": "y"}, inplace=True)

In [ ]:
cs.scatter_plot(
    sc_df,
    title="Scatter plot",
    xlabel="Tests", xlim=(0, None),
    ylabel="Confirmed",
    filename=None,
)

In [ ]:
with cs.ScatterPlot(filename=None) as sp:
    sp.plot(sc_df)
    sp.title = "Scatter plot"
    sp.x_axis(xlabel="Tests", xlim=(0, None))
    sp.y_axis(ylabel="Confirmed")
    sp.line_straight(p1=(0, 0), p2=(max(sc_df["x"]), max(sc_df["y"])))

Thank you!